# Computational Finance and Financial Management


## Time-series analysis, replication and forecasting of the German TecDAX stock index

By Merlin Bartel and Max Veltwisch

Trying to predict the stock market is an enticing prospect to us, not so much as a desire for material gain, but for the challenge. Already in 1965 the Economist and Nobel Price winner Paul Samuelson postulated in his influencial paper: "Properly Anticipated Prices Fluctuate Randomly" that returns of the past have no prediction power for future returns. He concluded that market prices could be considered a Martingale. This implies that the analysis and knownledge of historical price development is of no use for future price predictions. Nevertheless by looking at the daily up and downs of the market, we imagine there must be patterns we can model to beat all odds. Our analysis follows three main sections:

1. Exploratory analysis on the TecDAX Index price data
2. Applied Deep-Learning method to predict stock weights of the TecDAX index.
3. Forecasting the TecDAX price development using the Facebook Prophet model.
4. Forecasting the TecDAX price development using a LSTM model approach.

The first section serves the goal to better understand the general structure and charateristics of the historical TecDAX Index price development. Section two differs from the following sections by trying to estimate the weights of the stocks contained in the TecDAX Index in a way that replication becomes as precise as possible. The following sections make use of different model approaches with the goal of recognising patterns in the historical price movements of the TecDAX Index and using them to forecast future Index Prices.


Before we start with our research we load the required python packeges into our environment.

In [78]:
#loading required packages
import math
import pylab
import numpy as np
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from datetime import timedelta

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.losses import mean_squared_error
from keras import optimizers

import statistics
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from scipy import special, optimize

# from fbprophet import Prophet

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


## Exploratory analysis on the TecDAX index price data

We first want to analyse patterns and characteristics in the historically realized price development of the TecDAX Index. We are looking at the adjusted close prices of the TecDax Index from 01.01.2007 till 01.06.2019. The source of the TecDAX dataset and all following datasets is: finance.yahoo.com.

In [79]:
# # importing TecDAX Index price data
# TecDAX = pd.read_csv("TECDAX.csv", index_col=0, parse_dates=True, squeeze=True)
# # extracting adjusted close prices (variable of interest)
# TecDAX = TecDAX["Adj Close"]

In [80]:
# # showing general statisitcal values
# TecDAX.describe()

In [81]:
# # plotting historical price movement
# plt.style.use('fivethirtyeight')
# TecDAX.plot(figsize=(16,8), color='royalblue')
# plt.xlabel("Date",fontsize=18,color='black')
# plt.ylabel('Price',fontsize=18,color='black')
# plt.title("TecDAX Index",fontweight='bold',fontsize=22)
# plt.xticks(rotation=0)

# plt.show()

In [82]:
# # calculating the daily returns
# daily_returns = TecDAX.shift(1) / TecDAX - 1

# # visualising the daily returns
# daily_returns.plot(figsize=(16,8), color='royalblue')
# plt.xlabel("Date",fontsize=18,color='black')
# plt.ylabel("Returns",fontsize=18,color='black')
# plt.title("TecDAX Index daily returns", fontweight='bold',fontsize=22)
# plt.xticks(rotation=0)

# plt.show()

In [83]:
# # computing the first-difference
# training_diff = TecDAX.diff()

# # visualising first-difference
# training_diff.plot(figsize=(16,8), color='royalblue')
# plt.xlabel("Date",fontsize=18,color='black')
# plt.ylabel("Difference",fontsize=18,color='black')
# plt.title("TecDAX Index first difference", fontweight='bold',fontsize=22)
# plt.xticks(rotation=0)

# plt.show()

# Applied Deep-Learning method to predict stock weights of the TecDAX index. 


In [84]:
# Importing and indexing data
data = pd.read_csv("Alltogether.csv", encoding='latin-1')  #Adj. Closing prices of Yahoo, instead any other dataset can be used 
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)  

# Split TecDAX-Data from Stock-Data
y = data["TecDAX"]
y = y.values.reshape(len(y),1)
x = data.iloc [0:,1:]
x = x.values.reshape(len(y),41)

# Defining prediction horizon
test_size = 27
train_size  = len(y) - test_size

# Split training and test
y_train, y_test = y[0:train_size], y[train_size:len(y)]
x_train, x_test = x[0:train_size,:], x[train_size:len(y),:]


# Split Validation Set from training Set

# Define size
train_size2  = int(train_size * 0.7)

# Create Buffer-Variable to combine TecDAX with underlying stocks
buff = np.append(y_train, x_train, axis =1)

# Randomize Sample and Split TecDAX from Stocks again
np.random.shuffle(buff)
y_buff = buff[:,0]
x_buff = buff[:,1:]
np.reshape(x_buff, (len(x_buff),41))
np.reshape(y_buff, (len(x_buff),1))

# Rearrange original Data
y_train, y_valid = y_buff[0:train_size2], y_buff[train_size2:train_size]
x_train, x_valid = x_buff[0:train_size2,:], x_buff[train_size2:train_size,:]

In [85]:
# Build the neural network
def index_model():
    model = Sequential()
    
    # input layer
    model.add(Dense(1,activation='linear',input_dim=x_train.shape[1])) #the activation function can be linear, since the network consists of only 1 neuron
    
    # output layer
    model.add(Dense(1)) # Scalar Regression -> No Actication Function
    
    # Compile the model, define optimizer and loss function
    model.compile(optimizer='Adam',loss='mean_absolute_error') #We implemented the best choices of both in this code
    return model

In [ ]:
model = index_model()
# training the neural network, validation set is included to check if overfitting occurs
history = model.fit(x_train, y_train, epochs=20000, validation_data=(x_valid, y_valid)) # Storing all information during fitting in history variable
# geting the performance by using validaiton set 
Error = model.evaluate(x_train, y_train)
print("Error for the test set: ", Error)

Train on 685 samples, validate on 294 samples
Epoch 1/20000
685/685 [==============================] - 0s 525us/step - loss: 2186.0843 - val_loss: 2149.7347
Epoch 2/20000
685/685 [==============================] - 0s 68us/step - loss: 2151.7430 - val_loss: 2114.4192
Epoch 3/20000
685/685 [==============================] - 0s 68us/step - loss: 2114.7923 - val_loss: 2076.6960
Epoch 4/20000
685/685 [==============================] - 0s 46us/step - loss: 2075.5199 - val_loss: 2036.1158
Epoch 5/20000
685/685 [==============================] - 0s 46us/step - loss: 2033.2949 - val_loss: 1992.8669
Epoch 6/20000
685/685 [==============================] - 0s 68us/step - loss: 1988.4107 - val_loss: 1946.7510
Epoch 7/20000
685/685 [==============================] - 0s 68us/step - loss: 1940.4194 - val_loss: 1898.0397
Epoch 8/20000
685/685 [==============================] - 0s 69us/step - loss: 1889.9074 - val_loss: 1846.0871
Epoch 9/20000
685/685 [==============================] - 0s 68us/step - l

685/685 [==============================] - 0s 68us/step - loss: 21.8066 - val_loss: 24.9257
Epoch 77/20000
685/685 [==============================] - 0s 68us/step - loss: 21.7222 - val_loss: 24.6987
Epoch 78/20000
685/685 [==============================] - 0s 46us/step - loss: 21.6701 - val_loss: 24.7890
Epoch 79/20000
685/685 [==============================] - 0s 68us/step - loss: 21.6533 - val_loss: 24.6059
Epoch 80/20000
685/685 [==============================] - 0s 46us/step - loss: 21.4477 - val_loss: 24.6362
Epoch 81/20000
685/685 [==============================] - 0s 68us/step - loss: 21.5463 - val_loss: 24.4081
Epoch 82/20000
685/685 [==============================] - 0s 46us/step - loss: 21.2121 - val_loss: 24.1900
Epoch 83/20000
685/685 [==============================] - 0s 68us/step - loss: 21.1561 - val_loss: 24.1796
Epoch 84/20000
685/685 [==============================] - 0s 46us/step - loss: 21.1389 - val_loss: 23.9917
Epoch 85/20000
685/685 [============================

685/685 [==============================] - 0s 68us/step - loss: 16.8686 - val_loss: 18.9444
Epoch 153/20000
685/685 [==============================] - 0s 68us/step - loss: 16.7867 - val_loss: 19.3543
Epoch 154/20000
685/685 [==============================] - 0s 68us/step - loss: 16.6702 - val_loss: 19.0401
Epoch 155/20000
685/685 [==============================] - 0s 68us/step - loss: 16.7822 - val_loss: 18.6808
Epoch 156/20000
685/685 [==============================] - 0s 68us/step - loss: 16.5419 - val_loss: 18.6339
Epoch 157/20000
685/685 [==============================] - 0s 68us/step - loss: 16.6899 - val_loss: 18.6068
Epoch 158/20000
685/685 [==============================] - 0s 46us/step - loss: 16.5002 - val_loss: 18.5359
Epoch 159/20000
685/685 [==============================] - 0s 68us/step - loss: 16.5284 - val_loss: 18.5483
Epoch 160/20000
685/685 [==============================] - 0s 68us/step - loss: 16.3921 - val_loss: 18.6515
Epoch 161/20000
685/685 [===================

685/685 [==============================] - 0s 91us/step - loss: 14.9600 - val_loss: 16.9024
Epoch 229/20000
685/685 [==============================] - 0s 68us/step - loss: 14.6091 - val_loss: 16.0732
Epoch 230/20000
685/685 [==============================] - 0s 46us/step - loss: 14.2714 - val_loss: 16.8118
Epoch 231/20000
685/685 [==============================] - 0s 68us/step - loss: 14.6306 - val_loss: 16.2074
Epoch 232/20000
685/685 [==============================] - 0s 46us/step - loss: 14.4805 - val_loss: 15.8999
Epoch 233/20000
685/685 [==============================] - 0s 46us/step - loss: 14.2476 - val_loss: 15.7983
Epoch 234/20000
685/685 [==============================] - 0s 46us/step - loss: 14.3424 - val_loss: 15.7759
Epoch 235/20000
685/685 [==============================] - 0s 46us/step - loss: 14.1733 - val_loss: 15.9068
Epoch 236/20000
685/685 [==============================] - 0s 46us/step - loss: 14.1690 - val_loss: 15.7267
Epoch 237/20000
685/685 [===================

685/685 [==============================] - 0s 68us/step - loss: 12.8636 - val_loss: 14.8392
Epoch 305/20000
685/685 [==============================] - 0s 46us/step - loss: 13.2454 - val_loss: 14.1311
Epoch 306/20000
685/685 [==============================] - 0s 46us/step - loss: 12.6580 - val_loss: 14.1106
Epoch 307/20000
685/685 [==============================] - 0s 46us/step - loss: 12.7901 - val_loss: 14.0637
Epoch 308/20000
685/685 [==============================] - 0s 46us/step - loss: 12.6126 - val_loss: 14.0609
Epoch 309/20000
685/685 [==============================] - 0s 46us/step - loss: 12.5889 - val_loss: 14.0824
Epoch 310/20000
685/685 [==============================] - 0s 46us/step - loss: 12.6256 - val_loss: 14.0885
Epoch 311/20000
685/685 [==============================] - 0s 46us/step - loss: 12.5561 - val_loss: 13.9964
Epoch 312/20000
685/685 [==============================] - 0s 46us/step - loss: 12.5956 - val_loss: 14.2106
Epoch 313/20000
685/685 [===================

...

685/685 [==============================] - 0s 46us/step - loss: 4.5196 - val_loss: 4.8116
Epoch 14361/20000
685/685 [==============================] - 0s 46us/step - loss: 4.8820 - val_loss: 4.9732
Epoch 14362/20000
685/685 [==============================] - 0s 46us/step - loss: 4.7365 - val_loss: 5.3787
Epoch 14363/20000
685/685 [==============================] - 0s 68us/step - loss: 4.5716 - val_loss: 4.9536
Epoch 14364/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5481 - val_loss: 4.8755
Epoch 14365/20000
685/685 [==============================] - 0s 68us/step - loss: 4.4717 - val_loss: 4.9254
Epoch 14366/20000
685/685 [==============================] - 0s 46us/step - loss: 4.4989 - val_loss: 5.0114
Epoch 14367/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5385 - val_loss: 4.7995
Epoch 14368/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5067 - val_loss: 5.4971
Epoch 14369/20000
685/685 [===================

685/685 [==============================] - 0s 68us/step - loss: 4.5319 - val_loss: 5.1982
Epoch 14437/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6219 - val_loss: 5.0382
Epoch 14438/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5197 - val_loss: 4.7976
Epoch 14439/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5380 - val_loss: 4.8952
Epoch 14440/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6886 - val_loss: 4.7868
Epoch 14441/20000
685/685 [==============================] - 0s 68us/step - loss: 4.5074 - val_loss: 4.8411
Epoch 14442/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5125 - val_loss: 4.8317
Epoch 14443/20000
685/685 [==============================] - 0s 68us/step - loss: 4.6240 - val_loss: 4.8109
Epoch 14444/20000
685/685 [==============================] - 0s 68us/step - loss: 4.4728 - val_loss: 5.1586
Epoch 14445/20000
685/685 [===================

685/685 [==============================] - 0s 46us/step - loss: 4.5197 - val_loss: 4.8299
Epoch 14513/20000
685/685 [==============================] - 0s 68us/step - loss: 4.4915 - val_loss: 4.8008
Epoch 14514/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6454 - val_loss: 4.8414
Epoch 14515/20000
685/685 [==============================] - 0s 46us/step - loss: 4.4847 - val_loss: 4.8902
Epoch 14516/20000
685/685 [==============================] - 0s 46us/step - loss: 4.4906 - val_loss: 4.9268
Epoch 14517/20000
685/685 [==============================] - 0s 23us/step - loss: 4.6843 - val_loss: 5.6482
Epoch 14518/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5933 - val_loss: 5.0325
Epoch 14519/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5824 - val_loss: 4.7824
Epoch 14520/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5309 - val_loss: 4.9461
Epoch 14521/20000
685/685 [===================

685/685 [==============================] - 0s 68us/step - loss: 4.5613 - val_loss: 4.8958
Epoch 14589/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5164 - val_loss: 4.8888
Epoch 14590/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6995 - val_loss: 4.8489
Epoch 14591/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6298 - val_loss: 4.8344
Epoch 14592/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6052 - val_loss: 4.9136
Epoch 14593/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5278 - val_loss: 4.8545
Epoch 14594/20000
685/685 [==============================] - 0s 46us/step - loss: 4.4808 - val_loss: 4.8106
Epoch 14595/20000
685/685 [==============================] - 0s 46us/step - loss: 4.4828 - val_loss: 4.9470
Epoch 14596/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6555 - val_loss: 5.1001
Epoch 14597/20000
685/685 [===================

685/685 [==============================] - 0s 46us/step - loss: 4.6491 - val_loss: 4.8017
Epoch 14665/20000
685/685 [==============================] - 0s 68us/step - loss: 4.7120 - val_loss: 4.8130
Epoch 14666/20000
685/685 [==============================] - 0s 46us/step - loss: 4.6482 - val_loss: 4.8011
Epoch 14667/20000
685/685 [==============================] - 0s 68us/step - loss: 4.5580 - val_loss: 4.8238
Epoch 14668/20000
685/685 [==============================] - 0s 68us/step - loss: 4.4832 - val_loss: 4.8111
Epoch 14669/20000
685/685 [==============================] - 0s 46us/step - loss: 4.4887 - val_loss: 4.8013
Epoch 14670/20000
685/685 [==============================] - 0s 46us/step - loss: 4.4861 - val_loss: 4.7916
Epoch 14671/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5219 - val_loss: 4.8066
Epoch 14672/20000
685/685 [==============================] - 0s 46us/step - loss: 4.5211 - val_loss: 4.8435
Epoch 14673/20000
685/685 [===================

In [ ]:
# Visualizing Prediction vs Reality
predicted_level = model.predict(x_test)

plt.rcParams["figure.figsize"] = (16,8)
plt.plot(predicted_level, color = 'orangered', label = 'Prediction')
plt.plot(y_test[0:,], color = 'royalblue', label = 'TecDAX')
plt.title('TecDAX Index Prediction', fontweight='bold',fontsize=22)
plt.xlabel('Predicted days',fontsize=18,color='black')
plt.ylabel('Price',fontsize=18,color='black')
plt.legend()

plt.show()

In [ ]:
# visualising Overfitting
history_dict = history.history
history_dict.keys()
loss_values = np.log(history_dict['val_loss'])
acc = history_dict['val_loss']
epochs = range(1, len(acc) + 1)
plt.rcParams["figure.figsize"] = (16,8)
plt.plot(epochs, loss_values, color = 'orangered', label = 'Logarithmic Loss')
plt.title('Adam - log losses', fontweight='bold',fontsize=22)
plt.xlabel('Epochs',fontsize=18,color='black')
plt.ylabel('Loss',fontsize=18,color='black')
plt.legend()

plt.show()

# Forecasting the TecDAX price development using the Facebook Prophet model.

Not all forecasting problems can be solved by the same procedure. Prophet is optimized for business forecasting tasks, which have the following characteristics: hourly, daily, or weekly observations with at least a few months (preferably multiple years) of history. At its core, the Prophet procedure is an additive regression model with three main components:

 - A piecewise linear or logistic growth curve trend. Prophet automatically detects changes in trends by selecting changepoints from the data.
 - A yearly seasonal component modeled using Fourier series.
 - A weekly seasonal component using dummy variables.
 
We use the TecDAX Index dataset imported in the exploratory analysis section to train the prophet model. However we will not include the year 2018 and the first half of 2019 in the training set. After training the prophet model we predict the latest 1,5 year we didnt include in the training set. The prediction results will be shown as a solid blue line with an 80% confidence intervall(light blue area). In orange color we represent the true Index price realised during the year 2018 and the first half of 2019.
 

In [ ]:

def cycle_analysis(data,split_date,cycle,mode='additive',forecast_plot = False,print_ind=False):
    training = data[:split_date].iloc[:-1,]
    testing = data[split_date:]
    predict_period = len(pd.date_range(split_date,max(data.index)))
    df = training.reset_index()
    df.columns = ['ds','y']
    model = Prophet(weekly_seasonality=False,yearly_seasonality=False,daily_seasonality=False)
    model.add_seasonality('self_define_cycle',period=cycle,fourier_order=8,mode=mode)
    model.fit(df)
    future = model.make_future_dataframe(periods=predict_period)
    forecast = model.predict(future)
    if forecast_plot:
        model.plot(forecast)
        plt.plot(testing.index,testing.values,color='orangered',alpha=0.8)
        plt.title("TecDAX Index",fontweight='bold',fontsize=22)
        plt.xlabel('Date',fontsize=18,color='black')
        plt.ylabel('Price',fontsize=18,color='black')
        plt.show()
    ret = max(forecast.self_define_cycle)-min(forecast.self_define_cycle)
    model_tb = forecast['yhat']
    model_tb.index = forecast['ds'].map(lambda x:x.strftime("%Y-%m-%d"))
    return forecast

       
cycle_analysis(TecDAX,'2018-01-02',30,forecast_plot=True,print_ind=True)


# Forecasting the TecDAX price development using a LSTM model approach.

Again we use the TecDAX Index dataset from previous sections to train the prophet model. However this time we will not include the May 2019 in the training set. After training the prophet model we predict the month May that we didnt include in the training set. The prediction results will be shown as a solid orange line. In blue color we represent the true TecDAX Index price realised during the same month.

In [ ]:
# reshaping
TecDAX = TecDAX.values.reshape(len(TecDAX),1)

In [ ]:
# spliting the data into train and test data
train_size = int(len(TecDAX) * 0.993)
test_size  = len(TecDAX) - train_size

TecDAX_train, TecDAX_test = TecDAX[0:train_size,:], TecDAX[train_size:len(TecDAX),:]

print("Training observations: ", len(TecDAX_train))
print("Testing observations:  ", len(TecDAX_test))

In [ ]:
# normalizing the data
sc = MinMaxScaler(feature_range = (0, 1))

TecDAX_train.shape
training_set_scaled = sc.fit_transform(TecDAX_train)
test_set_scaled     = sc.fit_transform(TecDAX_test)

In [ ]:
# creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []

for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
#  !! fitting the model can take significant amount of time and computation power !!

# initialising the RNN
regressor = Sequential()

# adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
# adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
# adding the output layer
regressor.add(Dense(units = 1))
# compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
# getting the predicted stock price
dataset_total = pd.concat((pd.Series(TecDAX_train[:,0]), pd.Series(TecDAX_test[:,0])), axis = 0)
inputs = dataset_total[len(dataset_total) - len(TecDAX_test) - 60:]
inputs = inputs.values.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []

for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
    
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Visualising the results
plt.rcParams["figure.figsize"] = (16,8)
plt.plot(TecDAX_test[0:20,:], color = 'royalblue', label = 'TecDAX')
plt.plot(predicted_stock_price, color = 'orangered', label = 'Prediction')
plt.title('TecDAX Index Prediction', fontweight='bold',fontsize=22)
plt.xlabel('Predicted days',fontsize=18,color='black')
plt.ylabel('Price',fontsize=18,color='black')
plt.legend()

plt.show()